# pytorch使用GAN实现生成伪手写数字

## 预处理阶段

In [ ]:
# 导入包
import time
import sys
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

from torch.utils.tensorboard import SummaryWriter

In [ ]:
# 计划使用GPU运行
# 检查是否有可用的GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# 超参数设置
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [ ]:
# 建立文件夹用于存放训练过程的图像，如果文件夹不存在，就创建一个
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

## MNISR 数据集

In [ ]:
# transform配置
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=0.5,
            std=0.5,
        )
    ]
)

In [ ]:
# 加载mnist数据，同时将数据按照transform的配置预处理
# 判断是什么操作系统，以设置相应的数据集路径
if sys.platform.startswith('win'):
    datasetpath = 'D:\\nndatasets'
else:
    datasetpath = '/home/cxmd/文档/data_for_AI_train/mnist_data'

mnist = torchvision.datasets.MNIST(
    root=datasetpath,
    train=True,
    transform=transform,
    download=True,
)

In [ ]:
# 数据集加载器：GAN为无监督机器学习
# 参数dataset是要加载的数据集，这里是预处理后的mnist数据集
# 参数batch_size是每个批次的样本数量
# 参数shuffle为True表示在每个训练周期开始时，对数据进行重新洗牌
data_loader = torch.utils.data.DataLoader(
    dataset = mnist,
    batch_size=batch_size,
    shuffle=True,
)
data_loader

## 生成器和判别器的创建

In [ ]:
# 创建生成器
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

In [ ]:
# 创建判别器
D = nn.Sequential(
    nn.Linear(image_size, hidden_size), # 判别的输入时图像数据
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

In [ ]:
# 拷到计算设备上
D = D.to(device)
G = G.to(device)

In [ ]:
# 设置损失函数和优化器
criterion = nn.BCELoss() # 二值交叉熵 Binary cross entropy loss
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
# 定义两个函数
def denorm(x):
    # 使原本在[-1, 1]范围内的x变换到[0, 1]范围内
    out = (x + 1) / 2
    # 使用clamp函数确保所有的输出值都在[0, 1]范围内
    # 如果out中有小于0的值，它们会被设为0
    # 如果有大于1的值，它们会被设为1
    return out.clamp(0, 1)


# 重置梯度
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

## 训练

分两步  
1. 固定生成器，优化判别器
2. 固定判别器，优化生成器

In [ ]:
total_step = len(data_loader)   # 每次训练分为多少批次（训练集数量/batch_size大小）

# 记录开始训练的时间
start_time = time.time()

for epoch in range(num_epochs):      # 训练集总训练轮次
    for i, (images, labels) in enumerate(data_loader):   # 每次训练分为多少批次（训练集数量/batch_size大小）
        images = images.reshape(batch_size, -1).to(device)  # 目的是将所有图像压平成一维
        
        # 创建标签，随后会用于损失函数BCE loss的计算
        real_labels = torch.ones(batch_size, 1).to(device)    # 设置全1标签
        fake_labels = torch.zeros(batch_size, 1).to(device)   # 设置全0标签
        
        # ================================================================== #
        #                      训练判别模型                      #
        # ================================================================== #
        
        # 计算real损失
        # 使用公式 BCE_Loss(x, y) = -y * log(D(x)) - (1 - y) * log(1 - D(x)),来计算realimage的判别损失
        # 其中第二项永远为零，因为real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        
        # 计算fake损失
        # 生成器根据随机输入生成fake_images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        # 使用公式 BCE_Loss(x, y) = -y * log(D(x)) - (1 - y) * log(1 - D(x)),来计算fakeimage的判别损失
        # 其中第一项永远为零，因为fake_labels == 0
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 反向传播优化
        d_loss = d_loss_real + d_loss_fake     # 有个细节：本来要固定G，找到使损失最大时的D,但由于使用BCE_Loss函数计算时自动添加了负号，因此变为还是求损失函数的最小值，所以仍然可以通过后向传播更新参数
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                      训练生成模型                        #
        # ================================================================== #
        
        # 生成器根据随机输入生成fake_images，然后使用判别器进行判别
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # 训练生成模型，使之最大化 log(D(G(z)) ，而不是最小化 log(1-D(G(z)))
        # 具体的解释在原文第三小节最后一段有解释
        # 大致含义就是在训练初期，生成模型G还很菜，判别模型会以很高的置信度拒绝样本，因为这些样本与训练数据明显不同。
        # 这样log(1-D(G(z)))就近乎饱和，梯度计算得到的值很小，不利于反向传播和训练。
        # 换一种思路，通过计算最大化log(D(G(z))，就能够在训练初期提供较大的梯度值，利于快速收敛
        g_loss = criterion(outputs, real_labels)     # 相当于求-log(D(G(z)的最小值
        
        # 反向传播和优化
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i + 1) % 100 == 0:      # 每一百批次输入一次训练程度
            # 计算过去的时间
            elapsed_time = time.time() - start_time
            print('Elapsed time: {:.4f} seconds, Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'.format(elapsed_time, epoch + 1, num_epochs, i + 1, total_step, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))
    
    # 在第一轮保存训练数据图像
    if epoch == 0:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # 每一论保存生成的样本(集fake_images)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch + 1)))
    

## 结果展示

In [ ]:
# 导入包
import matplotlib.pyplot as plt
import scienceplots
plt.style.use(['science', 'ieee'])
import matplotlib.image as mpimg     # 用于读取图片
import numpy as np

real image

In [ ]:
real_path = './samples/real_images.png'
realImage = mpimg.imread(real_path)
plt.imshow(realImage)
plt.axis('off')   # 不显示坐标轴
plt.show()

#### fake image进化过程

In [ ]:
# 起始阶段
fakePath1 = './samples/fake_images-1.png'
fakeImg1 = mpimg.imread(fakePath1)

fakePath5 = './samples/fake_images-5.png'
fakeImg5 = mpimg.imread(fakePath5)


fig, ax = plt.subplots(1, 2)
ax[0].imshow(fakeImg1)
ax[1].imshow(fakeImg5)
plt.axis('off')
plt.show()


fakePath195 = './samples/fake_images-195.png'
fakeImg195 = mpimg.imread(fakePath195)

fakePath200 = './samples/fake_images-200.png'
fakeImg200 = mpimg.imread(fakePath200)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(fakeImg195)
ax[1].imshow(fakeImg200)
plt.axis('off')
plt.show()
